# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from workflow_widgets import WorkflowWidget, WorkerPoolWidget, TailWidget
from droplet_workflow import droplet_wf
from example_workflow import example_wf
from aux_widgets import QueueWidget, NEWTAuthWidget, SSHAuthWidget, SSHTerminal, Space
import workflow_objects as kale

import ipywidgets as ipw
import time
from concurrent.futures import ThreadPoolExecutor
import traitlets
import networkx as nx
import IPython.display as disp

In [3]:
wpw = WorkerPoolWidget()
wpw

A Jupyter Widget

In [5]:
ww = WorkflowWidget(workflow=droplet_wf, worker_pool_widget=wpw)
ww

A Jupyter Widget

Not removing 41 from 60A
Not removing 42 from 60A
Not removing 43 from 60A
Not removing 44 from 60A
Not removing 45 from 60A
Not removing 46 from 60A
Not removing 47 from 60A
Not removing 48 from 60A
Not removing 49 from 60A
Not removing 50 from 60A
Adding 41 to 60A
Adding 21 to 60A
Adding 42 to 60A
Adding 22 to 60A
Adding 43 to 60A
Adding 23 to 60A
Adding 48 to 60A
Adding 24 to 60A
Adding 46 to 60A
Adding 28 to 60A
Adding 44 to 60A
Adding 26 to 60A
Adding 45 to 60A
Adding 27 to 60A
Adding 25 to 60A
Adding 49 to 60A
Adding 47 to 60A
Adding 29 to 60A
Adding 50 to 60A
Adding 30 to 60A
Adding 61 to 60A
Adding 62 to 60A


TypeError: 'NoneType' object is not iterable

# Define simple workflow

In [ ]:
wfw = WorkflowWidget(example_wf, wpw)
wfw

In [ ]:
example_workflow = kale.Workflow(name='example')
for i in range(10):
    example_workflow.add_task(
        kale.CommandLineTask(
            name='echo_test_{num}',
            command='echo "Test #{num}" >> {out_file} &&',
            tags=['run'],
            output_files=["{out_file}"],
            params=dict(
                num=i,
                out_file="output.txt"
            )
        )
    )

In [ ]:
wfw.workflow.tag_dict

In [ ]:
wfw.workflow._new_tag('newtag')

In [ ]:
wfw._update_tag_selector()

# Connect to Cluster


In [ ]:
ssh = SSHAuthWidget(host='cori.nersc.gov', username='oevans')
ssh

In [ ]:
t = SSHTerminal(ssh, width=800)
t

# Create Worker Pool


# Visualize & Run Workflow

In [ ]:
exww.workflow.tag_dict

In [ ]:
selected = exww.bqgraph.selected
tasks = [exww.bqgraph.node_data[x]['name'] for x in selected]
[exww.workflow.get_task_by_name(name) for name in tasks]

In [ ]:
nx.dfs_preorder_nodes?

In [ ]:
exww.workflow.dag.o(node)

In [ ]:
exww.bqgraph.selected = list(range(5))

In [ ]:
example_wf.dag.nodes()[1].children

# Observe Output

In [ ]:
t = TailWidget('out.txt')
t

# Connect to NEWT

In [ ]:
a = NEWTAuthWidget()
a

In [ ]:
q = QueueWidget(a)
q